In [81]:
!pip install chromium

ERROR: Could not find a version that satisfies the requirement chromium (from versions: none)
ERROR: No matching distribution found for chromium
You should consider upgrading via the 'C:\Users\Sveta\projects\недвижка парсер\env\Scripts\python.exe -m pip install --upgrade pip' command.


In [82]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd
from tqdm import tqdm
import numpy as np
# driver = webdriver.Chrome()

from selenium.webdriver.chrome.options import Options
options = Options()
options.binary_location = r"C:\Program Files\Google\Chrome Beta\Application\chrome.exe"
driver = webdriver.Chrome(chrome_options = options, executable_path='chromedriver.exe')

C:\Users\Sveta\AppData\Local\Temp\ipykernel_15104\651899051.py:11: DeprecationWarning: executable_path has been deprecated, please pass in a Service object
  driver = webdriver.Chrome(chrome_options = options, executable_path='chromedriver.exe')
C:\Users\Sveta\AppData\Local\Temp\ipykernel_15104\651899051.py:11: DeprecationWarning: use options instead of chrome_options
  driver = webdriver.Chrome(chrome_options = options, executable_path='chromedriver.exe')


In [83]:
def get_html(url, PARAMS = None): 
    html = driver.page_source
    return html

def get_content(html): 
    soup = BeautifulSoup(html, 'html.parser')
    items = soup.find_all('article', class_ = '_93444fe79c--container--Povoi')
    return items
    
def get_urls():   
    urls = []
    params = ['&sort=creation_date_desc', 
              '&sort=creation_date_asc', 
              '&sort=price_object_order', 
              '&sort=total_price_desc', 
              '&sort=street_name', 
              '&sort=walking_time']
    
    for par in tqdm(range(len(params))):
        param = params[par] 
        for i in range(1,55): 
            URL = driver.get(f'https://www.cian.ru/cat.php?deal_type=rent&engine_version=2&offer_type=flat&p={i}&region=1&room1=1&{param}&type=4')
            html = get_html(URL)
            items = get_content(html)
            for item in items:
                urls.append(item.find('a', class_='_93444fe79c--link--eoxce').get('href'))
        urls = list(set(urls))
    return urls

In [84]:
def get_data(urls): 
    prices, metro, floor, max_floor, size, msize, deadline, bathroom, renovation, time = [], [], [], [], [], [], [], [], [], []
    for i in tqdm(range(len(urls))):
        url = urls[i]
        URL = driver.get(url)
        html = get_html(URL)
        soup = BeautifulSoup(html, 'html.parser')
        try: 
            price = soup.find('span', class_ = "a10a3f92e9--price_value--lqIK0").find('span').get('content')
            prices.append(int(price.replace(' ', '').rstrip('₽/мес.')))
        except AttributeError: prices.append(np.nan)
            
        try: metro.append(soup.find('a', class_ = 'a10a3f92e9--underground_link--Sxo7K').get_text())
        except AttributeError: metro.append(np.nan)
           
        try: time.append(int(soup.find('span', class_ = 'a10a3f92e9--underground_time--iOoHy').get_text()[4:6]))
        except AttributeError: time.append(np.nan)
        except ValueError: time.append(np.nan)
         
        block1 = soup.find_all('div', class_ = 'a10a3f92e9--info-value--bm3DC')
        titles1 = soup.find_all('div', class_ = 'a10a3f92e9--info-title--JWtIm')
        
        flag = [0,0,0,0]
        for i in range(len(titles1)):
            if titles1[i].get_text() == 'Общая':
                flag[0] = 1
                try: size.append(float(block1[i].get_text().rstrip('\xa0м²').replace(',','.')))
                except ValueError: size.append(np.nan)
            elif titles1[i].get_text() == 'Жилая':
                flag[1] = 1
                try: msize.append(float(block1[i].get_text().rstrip('\xa0м²').replace(',','.')))
                except ValueError: msize.append(np.nan)
            elif titles1[i].get_text() == 'Этаж':
                flag[2] = 1
                floors = block1[i].get_text().split(' из ')
                floor.append(int(floors[0]))
                max_floor.append(int(floors[1]))
            elif titles1[i].get_text() == 'Построен' or titles1[i].get_text() == 'Срок сдачи':
                flag[3] = 1
                deadline.append(int(block1[i].get_text()[-4:]))
        if flag[0] == 0:
            size.append(np.nan)
        if flag[1] == 0:
             msize.append(np.nan)
        if flag[2] == 0:
            floor.append(np.nan)
            max_floor.append(np.nan)
        if flag[3] == 0:
            deadline.append(np.nan)

        block2 = soup.find_all('span', class_ = 'a10a3f92e9--value--Y34zN')
        titles2 = soup.find_all('span', class_ = 'a10a3f92e9--name--x7_lt')

        flag = [0,0,0,0]
        for i in range(len(titles2)):
            if titles2[i].get_text() == 'Санузел':
                flag[0] = 1
                bathroom.append(block2[i].get_text())
            elif titles2[i].get_text() == 'Ремонт':
                flag[1] = 1
                renovation.append(block2[i].get_text())
        if flag[0] == 0:
            bathroom.append(np.nan)
        if flag[1] == 0:
            renovation.append(np.nan)
            
        
    return prices, metro, size, msize, floor, max_floor, deadline, bathroom, renovation, time

In [85]:
urls = get_urls()

100%|██████████| 6/6 [14:48<00:00, 148.16s/it]


In [86]:
prices, metro, size, msize, floor, max_floor, deadline, bathroom, renovation, time = get_data(urls)

100%|██████████| 5916/5916 [3:54:24<00:00,  2.38s/it]   


In [87]:
df = pd.DataFrame({ 
    'Ссылка' : urls, 
    'Метро' : metro, 
    'Время до метро' : time,
    'Общая площадь' : size, 
    'Жилая площадь' : msize, 
    'Этаж' : floor,
    'Этажность дома' : max_floor,
    'Срок сдачи' : deadline,
    'Ванная' : bathroom,
    'Ремонт': renovation,
    'Цена' : prices
    })

In [88]:
df.to_csv('cian.csv')